# Data Download

In [1]:
import ccxt
import pandas as pd
import time

# Initialize the Binance API
binance = ccxt.binance()

# Define the symbol and timeframe
symbol = "XRP/USDT"
timeframe = "1m"  # 1 minute

# Define the date range
since = binance.parse8601("2020-09-01T00:00:00Z")
end = binance.parse8601("2022-09-01T00:00:00Z")

# Check if the CSV file already exists.
# If it does, read the last timestamp to avoid downloading duplicate data.
try:
    existing_df = pd.read_csv(f"year_{symbol[0:3]}_historical_data_1m.csv")
    # since = pd.to_datetime(existing_df["timestamp"]).max().value // 10**6
    print(f"Resuming from timestamp: {since}")
except FileNotFoundError:
    # Initialize an empty DataFrame to store the data if the file doesn't exist
    df = pd.DataFrame(
        columns=["timestamp", "open", "high", "low", "close", "volume", "symbol"]
    )
    df.to_csv(f"year_{symbol[0:3]}_historical_data_1m.csv", index=False)

# Loop to paginate through the data
while since < end:
    print(f"Since: {since}")
    # Download a chunk of historical data
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, since)

    # If no new data, break the loop
    if len(ohlcv) == 0:
        break

    # Convert the chunk to a DataFrame
    chunk_df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )

    # Convert the timestamp to a readable date format
    chunk_df["timestamp"] = pd.to_datetime(chunk_df["timestamp"], unit="ms")

    chunk_df["symbol"] = symbol

    # Append the chunk to the CSV file
    chunk_df.to_csv(
        f"year_{symbol[0:3]}_historical_data_1m.csv",
        mode="a",
        header=False,
        index=False,
    )

    # Update the 'since' parameter for the next iteration
    since = ohlcv[-1][0] + 1  # Timestamp of the last entry + 1 millisecond

    # Sleep for a while to avoid hitting the API rate limit
    time.sleep(1)

Since: 1598918400000
Since: 1598948340001
Since: 1598978340001
Since: 1599008340001
Since: 1599038340001
Since: 1599068340001
Since: 1599098340001
Since: 1599128340001


KeyboardInterrupt: 